In [11]:
import sqlite3
import pandas as pd

Conectamos a la base de Datos con SQL lite

In [12]:
conexion=sqlite3.connect("database.db")
cursor=conexion.cursor()

Nivel 1 – Exploración básica (SELECT, LIMIT, DISTINCT, WHERE)

In [13]:
#¿Cuáles son las primeras 10 observaciones registradas?
con=cursor.execute("SELECT * FROM observations LIMIT 10")
#Creamos un DF
filas=con.fetchall()
columnas=[description[0]for description in con.description]

df=pd.DataFrame(filas,columns=columnas)
df

,id,species_id,region_id,observer,observation_date,latitude,longitude,count
0,1,1,1,obsr297898,1992-07-07,-16.820152,145.63625,0
1,2,2,2,obsr1375321,1997-08-08,-33.126945,151.61362,0
2,3,3,1,obsr1898605,1981-12-13,-20.594380,139.54472,0
3,4,4,2,obsr166621,1999-10-17,-34.652440,150.80241,0
4,5,2,2,obsr797550,1999-11-13,-33.740300,151.08450,0
5,6,5,3,obsr340454,1997-05-10,-37.997130,144.56042,10
6,7,6,2,obsr1375321,1973-10-22,-37.439445,149.96638,15
7,8,7,3,obsr1437445,1995-01-07,-37.785960,145.01482,0
8,9,8,2,obsr631796,1966-12-10,-33.813310,150.90550,0
9,10,9,3,obsr2831785,1986-12-27,-37.227900,147.92300,0


In [14]:
#¿Qué identificadores de región ( region_id) aparecen en los datos?
con=cursor.execute("SELECT DISTINCT region_id FROM observations")
fila=con.fetchall()
columna=[decription[0] for decription in con.description]
df=pd.DataFrame(fila,columns=columna)
df

,region_id
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [15]:
#¿Cuántas especies distintas ( species_id) se han observado?
con=cursor.execute("SELECT COUNT(DISTINCT species_id) AS total_species FROM observations")
valor=con.fetchone()[0]

print(f"se han observado {valor} especies distintas")

se han observado 270 especies distintas


In [16]:
#¿Cuántas observaciones hay para la región con region_id = 2?
con=cursor.execute("SELECT *  FROM observations WHERE region_id = 2")
filas=con.fetchall()
columnas=[description[0] for description in con.description]
df=pd.DataFrame(filas,columns=columnas)
print(df)
print(f"Hay un total de {df.shape[0]} observacions para la region con region_id=2")

      id  species_id  region_id     observer observation_date   latitude  \
0      2           2          2  obsr1375321       1997-08-08 -33.126945   
1      4           4          2   obsr166621       1999-10-17 -34.652440   
2      5           2          2   obsr797550       1999-11-13 -33.740300   
3      7           6          2  obsr1375321       1973-10-22 -37.439445   
4      9           8          2   obsr631796       1966-12-10 -33.813310   
..   ...         ...        ...          ...              ...        ...   
223  493          49          2   obsr450923       1992-06-30 -28.628270   
224  494          88          2  obsr1547001       1988-03-14 -33.843758   
225  495           9          2    obsr45447       1986-10-25 -30.394480   
226  497          49          2   obsr453532       1979-11-23 -32.176884   
227  499         183          2   obsr629213       1983-03-05 -33.614727   

     longitude  count  
0    151.61362      0  
1    150.80241      0  
2    151.08450 

In [17]:
#¿Cuántas observaciones se registraron el día 1998-08-08?
con=cursor.execute("""
SELECT * 
FROM observations 
WHERE observation_date = '1998-08-08'""") 
fila=con.fetchall()
columnas=[description[0] for description in con.description]
df=pd.DataFrame(fila,columns=columnas)
df

,id,species_id,region_id,observer,observation_date,latitude,longitude,count
0,496,79,1,obsr202543,1998-08-08,-17.285713,145.62866,0


In [18]:
con.close()

Nivel 2 – Agregación y ordenamiento (GROUP BY, COUNT, ORDER BY, HAVING sin JOIN)

In [21]:
#¿Cuál es el region_id con más observaciones?
con = sqlite3.connect("database.db")
query = """
SELECT region_id, 
COUNT(*) 
AS registros
FROM observations
GROUP BY region_id 
ORDER BY registros DESC
LIMIT 10
"""
df = pd.read_sql(query, con)
print(df)

   region_id  registros
0          2        228
1          1         81
2          3         50
3          7         45
4          9         20
5         14         11
6          8         10
7         19          6
8         16          6
9         13          6


In [20]:
#¿Cuáles son los 5 species_id más frecuentes?
query="""
SELECT species_id,
COUNT (*)
AS registros
FROM observations
GROUP BY species_id
ORDER BY frecuencia DESC
LIMIT 5"""
df = pd.read_sql(query,con)
print(df)

DatabaseError: Execution failed on sql '
SELECT species_id,
COUNT (*)
AS registros
FROM observations
GROUP BY species_id
ORDER BY frecuencia DESC
LIMIT 5': no such column: frecuencia

In [ ]:
#¿Qué especies ( species_id) tienen menos de 5 registros?
query="""
SELECT species_id,
COUNT (*)
AS registros
FROM observations
GROUP BY species_id
HAVING COUNT (*) < 5
"""
df=pd.read_sql(query,con)
df

,species_id,frecuencia
0,2,4
1,3,1
2,5,2
3,6,1
4,7,3
...,...,...
244,266,1
245,267,1
246,268,2
247,269,1


In [ ]:
#¿Qué observadores ( observer) registraron más observaciones?
query="""
SELECT observer,
COUNT (*)
AS registros
FROM observations
GROUP BY observer
ORDER BY registros DESC
"""
df=pd.read_sql(query,con)
df

,observer,registros
0,obsr453532,72
1,obsr1375321,20
2,obsr450704,15
3,obsr450476,13
4,obsr438330,13
...,...,...
186,obsr131036,1
187,obsr1169933,1
188,obsr1164766,1
189,obsr115435,1


Nivel 3 – Relaciones entre tablas (ÚNETE)